In [ ]:
pip install faiss-cpu

In [ ]:
เริ่มจากload environmentและตั้งค่าAPI_KEYก่อน

In [ ]:
import os
from dotenv import load_dotenv

# load environment
load_dotenv(dotenv_path=r"my_directory\HUGGINGFACE_API_KEY.env")

# ตั้งค่า API Key ของ Hugging Face
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY") # ใส่คีย์ API ของคุณที่นี่

if "HUGGINGFACE_API_KEY" in os.environ:
    print("Environment variable loaded successfully!")
    print("HUGGINGFACE_API_KEY:", os.environ["HUGGINGFACE_API_KEY"])  # ตรวจสอบค่า (อย่าแชร์ค่า API Key จริง)
else:
    print("Failed to load environment variable.")


ทำการโหลดโมเดลLLMที่ต้องการใช้งานเข้ามา พร้อมทั้งระบุหน้าที่ของโมเดล

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="meta-llama/Llama-3.2-1B")

ใช้WebBaseLoaderและSoupStrainerในการทำweb scraping ดึงข้อมูลจากเว็บไซต์ที่ต้องการและเก็บไว้ในตัวแปรdocs

In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

ใช้ RecursiveCharacterTextSplitterเพื่อ
1.ใช้แบ่งเอกสารขนาดใหญ่ให้เหมาะสมกับ Embeddings Models (เช่น OpenAI, Hugging Face)
2.ใช้แบ่งข้อความก่อนนำไปประมวลผลกับ LLMs (Large Language Models)
3.ใช้เตรียมข้อมูลก่อนนำเข้า Vector Database เช่น FAISS, Chroma

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)
splits = text_splitter.split_documents(docs)
print(splits)

ขั้นตอนต่อไปคือการนำ Chunks ข้อมูลเหล่านั้นมาเข้าสู่กระบวนการ Embeddings เพื่อแปลงข้อมูลให้อยู่ในรูปแบบของ Vector หลังจากนั้นเราจะเก็บข้อมูล Vectors ที่ได้ไว้ใน FAISS ซึ่งเป็น Vector store library ที่ถูกพัฒนามาสำหรับการทำ Similarity Search อย่างมีประสิทธิภาพ

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

#HuggingFaceEmbeddings
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# storing embeddings in the vector store
vectorstore = FAISS.from_documents(splits, embeddings)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

ต่อมา เราจะสร้างpromptเพื่อช่วยให้โมเดลเข้าใจบริบทผ่าน {context} ซึ่งcontextคือข้อมูลที่เราจะดึงมาจาก FAISS Vectorstore

In [ ]:
#create function for easing to create prompt biolderpalte
def make_llama_3_prompt(question, system="", context=""):
  system_prompt = ""
  if system != "":
    system_prompt = (
        f"<|start_header_id|>system<|end_header_id|>\n\n{system}\n\n"
        f"context: {context}\n\n"
        f"<|eot_id|>\n\n"
    )
  prompt = (
      f"<|begin_of_text|>{system_prompt}"
      f"<|start_header_id|>user<|end_header_id|>\n\n"
      f"{question}\n\n"
      f"<|eot_id|>\n\n"
      f"<|start_header_id|>assistant<|end_header_id|>\n\n" # header - assistant
  )
  return prompt

system_prompt = "You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise."
context = "filler context"
user_prompt = "What is Task Decomposition?"
prompt3 = make_llama_3_prompt(user_prompt, system_prompt, context)
print(prompt3)

แต่เนื่องจากเราจะใช้ความสามารถของ LangChain Expression Language (LCEL) ในการทำ pipeline ด้วยการ chain ขั้นตอนต่างๆเข้าด้วย โดยให้ output ที่ได้จาก process ก่อนหน้า ไปเป็น input ให้กับ process ถัดไปเพื่อทำงานต่อแบบนี้ไปเรื่อยๆจนจบ process ดังนั้น เราต้องแปลง Function ด้านบนให้อยู่ในรูปแบบของ class ที่ implement Runnable interface เช่นเดียวกับ retriever และ llm โดยเราจะ implement method .invoke เพื่อให้เราสามารถ chain process เข้าหากันได้ด้วยความสามารถของ LCEL

In [ ]:
# Meta Llama 3 Instruct uses a prompt template, with special tags used to indicate the user query and system prompt.
# You can find the documentation on this [model card](https://llama.meta.com/docs/model-cards-and-prompt-formats/meta-llama-3/#meta-llama-3-instruct).

from langchain_core.runnables import Runnable
class Llama3PromptRunnable(Runnable):
    def __init__(self, system=""):
        super().__init__()
        self.system = system

    def invoke(self, inputs: dict, config=None) -> str:
        question = inputs["question"]
        context = inputs["context"]
        # Create the system prompt if provided
        system_prompt = ""
        if self.system != "":
            system_prompt = (
                f"<|start_header_id|>system<|end_header_id|>\n\n{self.system}\n\n"
                f"context: {context}\n\n"
                f"<|eot_id|>\n\n"
            )
            prompt = (
                f"<|begin_of_text|>{system_prompt}"
                f"<|start_header_id|>user<|end_header_id|>\n\n"
                f"{question}\n\n"
                f"<|eot_id|>\n\n"
                f"<|start_header_id|>assistant<|end_header_id|>\n\n" # header - assistant
            )

        # Return the formatted prompt
        return prompt

In [ ]:
# Example usage
llama_prompt = Llama3PromptRunnable(system="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.")
formatted_prompt = llama_prompt.invoke({"context": "filler context", "question": "What is Task Decomposition?"})
print(formatted_prompt)

หลังจากที่เราเตรียมทุกอย่างพร้อมแล้ว ไม่ว่าจะเป็น llm, retrieval, และ prompt ขั้นตอนถัดไปเราจะร้อยสามสิ่งนี้เข้าด้วยกัน ด้วยความสามารถของ LCEL Runnable โดยเราต้องใช้RunnableLambdaเพื่อดึงคำตอบที่โมเดลเจนออกมาแล้วส่งผ่านStrOutputParserเพื่อช่วยจัดการกับคำตอบให้หน้าตาเข้าถึงง่ายขึ้น

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()} #output {"context":"XX", "question": "YY"}
    | llama_prompt #take input from previous runable, {"context":"XX", "question": "YY"}, and return prompt str
    | pipe #take input from previous runable, prompt str, to generate the answer
    | RunnableLambda(lambda x: x[0]["generated_text"])  # ดึงค่า generated_text ออกมา
    | StrOutputParser() #take input from previous runable, answer, to parser output
)

rag_chain.invoke("What is Task Decomposition?")

ต่อมาเราจะสร้างHistory chatเพื่อให้โมเดลสามารถจดจำcontextของคำถามเดิมได้

In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.chains import ConversationalRetrievalChain

llm = HuggingFacePipeline(pipeline=pipe)  # แปลงเป็น Runnable
chain = ConversationalRetrievalChain.from_llm(llm, retriever, return_source_documents=True)

chat_history = []

query = "What is Task Decomposition?"
response = chain({"question": query, "chat_history": chat_history})

print(response['answer'])

ทีนี้เราลองถามต่อจาก context เดิมดังนี้

In [ ]:
chat_history = [(query, response["answer"])]

query = "What are the challenges?"
result = chain({"question": query, "chat_history": chat_history})

print(result['answer'])